In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

In [2]:
import pandas as pd
import datasets

patient_notes = pd.read_csv('data/en/patient_notes.csv')
patient_notes.head()

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [4]:
from tools.text_tools import process_en

patient_notes.apply(lambda x: process_en(x['pn_history']), axis=1)
# patient_notes.values[0][2].replace('-year-old', ' year old ')

0        17 year old male, has come to the student heal...
1        17 year old male with recurrent palpitations f...
2        Dillon Cleveland is a 17 year old male patient...
3        a 17 year old m c/o palpitation started 3 mos ...
4        17yo male with no pmh here for evaluation of p...
                               ...                        
42141    Ms. Madden is a 20 year old female presenting ...
42142    A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...
42143    Ms. Madden is a 20yo female who presents with ...
42144    Stephanie madden is a 20 year old woman compla...
42145    patient is a 20 year old F who presents with a...
Length: 42146, dtype: object